## TF

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import mnist
import matplotlib.pylab as plt

import numpy as np
import os

## Parameter Setting

In [ ]:
learning_rate = 0.001
epochs = 20
batch_size=64

## Load Data

In [ ]:
(train_image, train_label), (test_image, test_label) = mnist.load_data()

train_image = train_image.astype(np.float32)
test_image = test_image.astype(np.float32)

# train_image = np.expand_dims(train_image, axis=1)
# test_image = np.expand_dims(test_image, axis=1)

train_label = to_categorical(train_label, 10)
test_label = to_categorical(test_label, 10)

train_data = tf.data.Dataset.from_tensor_slices((train_image, train_label)).shuffle(buffer_size=1000000).batch(batch_size)
test_data = tf.data.Dataset.from_tensor_slices((test_image, test_label)).batch(batch_size)

In [ ]:
img = train_image[0].reshape([28,28])
print(type(img))
print(img.shape)
plt.figure(figsize=(8,8))
plt.imshow(img, cmap='gray_r')
plt.show()

## Model setting

In [ ]:
ip = keras.Input(shape=(28,28,1))

# First Conv
conv1 = keras.layers.Conv2D(filters=64, kernel_size=(3,3), strides=1, padding='same', activation=tf.nn.relu)(ip)
pool1 = keras.layers.MaxPool2D(pool_size=(2,2), padding='same')(conv1)
norm1 = tf.nn.local_response_normalization(pool1, depth_radius=4, bias=1.0, alpha=0.001/9.0, beta=0.75)
drop1 = keras.layers.Dropout(0.8)(norm1)

# Second Conv
conv2 = keras.layers.Conv2D(filters=128, kernel_size=(3,3), strides=1, padding='same', activation=tf.nn.relu)(drop1)
pool2 = keras.layers.MaxPool2D(pool_size=(2,2), padding='same')(conv2)
norm2 = tf.nn.local_response_normalization(pool2, depth_radius=4, bias=1.0, alpha=0.001/9.0, beta=0.75)
drop2 = keras.layers.Dropout(0.8)(norm2)

# Third Conv
conv3 = keras.layers.Conv2D(filters=256, kernel_size=(3,3), strides=1, padding='same', activation=tf.nn.relu)(drop2)
pool3 = keras.layers.MaxPool2D(pool_size=(2,2), padding='same')(conv3)
norm3 = tf.nn.local_response_normalization(pool3, depth_radius=4, bias=1.0, alpha=0.001/9.0, beta=0.75)
drop3 = keras.layers.Dropout(0.8)(norm3)

# Fully Connected
flat = keras.layers.Flatten()(drop3)
dense1 = keras.layers.Dense(units = 1024, activation = tf.nn.relu)(flat)
dense2 = keras.layers.Dense(units = 1024, activation = tf.nn.relu)(dense1)

# Output
op = keras.layers.Dense(units=10)(dense2)
model = keras.Model(ip, op)

## Fit

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
history = model.fit(train_image, train_label, epochs=epochs)

## Graph

## ETC